In [1]:
import numpy as np
import pandas as pd

import ast

In [2]:
link_detail = pd.read_csv('link_detail.csv')
link_type = pd.read_csv('link_type.csv', index_col=False)
tthc_detail = pd.read_csv('tthc_detail.csv', dtype={'maThuTuc': str})
tthc_recrawl = pd.read_csv('tthc_recrawl.csv')

In [3]:
(link_detail['link'] == link_type['link']).sum() == len(link_type) == len(link_detail)

True

In [4]:
link_detail.duplicated().sum()

np.int64(0)

In [5]:
link_raw = pd.concat([link_type, link_detail.loc[:, ~link_detail.columns.isin(link_type.columns)]], axis=1)
link_raw['TTHCLienQuan'] = link_raw['TTHCLienQuan'].apply(ast.literal_eval)
link_raw['cauHoiLienQuan'] = link_raw['cauHoiLienQuan'].apply(ast.literal_eval)
link_raw.sample()

,link,phanLoai,boNganh,cauHoi,cauTraLoi,TTHCLienQuan,cauHoiLienQuan
9029,https://dichvucong.gov.vn/p/home/dvc-chi-tiet-...,[],['Bộ Nông nghiệp và Môi trường'],Đề nghị cho biết thời gian xem xét hồ sơ đề ng...,"Theo quy định tại khoản 1, điểm a-khoản2, Điều...",[https://dichvucong.gov.vn/p/home/dvc-chi-tiet...,[https://dichvucong.gov.vn/p/home/dvc-chi-tiet...


In [6]:
tthc_recrawl.info()
tthc_detail.maThuTuc.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81 entries, 0 to 80
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   link                    81 non-null     object
 1   trinhTuThucHien         81 non-null     object
 2   cachThucThucHien        81 non-null     object
 3   thanhPhanHoSo           81 non-null     object
 4   coQuanThucHien          81 non-null     object
 5   yeuCauDieuKienThucHien  22 non-null     object
dtypes: object(6)
memory usage: 3.9+ KB


np.int64(89)

Có một dòng là `Không có thông tin` suy ra `89+1-9=81`

In [7]:
tthc_recrawl['maThuTuc'] = tthc_recrawl['link'].str.split('ma_thu_tuc=').str[-1].astype('str')
filling_cols = ['maThuTuc', 'trinhTuThucHien', 'cachThucThucHien', 'thanhPhanHoSo', 'coQuanThucHien', 'yeuCauDieuKienThucHien']
for i in range(len(tthc_detail)):
    if tthc_detail['link'][i] in tthc_recrawl['link'].to_list():
        tthc_detail.loc[i, filling_cols] = tthc_recrawl[tthc_recrawl['link']==tthc_detail['link'][i]][filling_cols].iloc[0].tolist()
        
tthc_detail.maThuTuc.isna().sum()

np.int64(9)

In [8]:
tthc_detail[tthc_detail['maThuTuc'].isna()].link == tthc_detail[~tthc_detail['link'].str.contains('ma_thu_tuc')].link

1920    True
1921    True
1922    True
1923    True
1924    True
1925    True
1926    True
1927    True
2265    True
Name: link, dtype: bool

In [9]:
error_list = tthc_detail[~tthc_detail['link'].str.contains('ma_thu_tuc')].link.to_list()
tthc_raw = tthc_detail[~tthc_detail['maThuTuc'].isna()].reset_index(drop=True)
tthc_raw.tail(1)

,link,maThuTuc,soQuyetDinh,tenThuTuc,capThucHien,loaiThuTuc,linhVuc,trinhTuThucHien,cachThucThucHien,thanhPhanHoSo,...,coQuanThucHien,coQuanCoThamQuyen,diaChiTiepNhanHoSo,coQuanDuocUyQuyen,coQuanPhoiHop,ketQuaThucHien,canCuPhapLy,yeuCauDieuKienThucHien,tuKhoa,moTa
2694,https://dichvucong.gov.vn/p/home/dvc-chi-tiet-...,2.000895,1792 /QĐ-BKHCN,"Thủ tục đánh giá, nghiệm thu và công nhận kết ...",Cấp Tỉnh,TTHC không được luật giao cho địa phương quy đ...,Sở hữu trí tuệ,- Bước 1: Tổ chức chủ trì và chủ nhiệm dự án t...,"Hình thức nộp Thời hạn giải quyết Phí, lệ phí ...","Bao gồm\nTên giấy tờ Mẫu đơn, tờ khai Số lượng...",...,Sở Khoa học và Công nghệ,Không có thông tin,Sở Khoa học và Công nghệ,Không có thông tin,Không có thông tin,Quyết định công nhận kết quả thực hiện dự án c...,Số ký hiệu Trích yếu Ngày ban hành Cơ quan ban...,Không,Không có thông tin,Không có thông tin


In [10]:
temp_list = link_raw['cauHoiLienQuan'].explode().dropna().drop_duplicates().to_list()
for i in temp_list:
    if i not in link_raw['link'].to_list():
        print(i)

In [11]:
def saveOnlyTTHC(s):
    return [i for i in s if i not in error_list]
link_raw['TTHCLienQuan'] = link_raw['TTHCLienQuan'].apply(saveOnlyTTHC)

In [12]:
link_raw.to_csv('raw_data/raw_link.csv', index=False)
tthc_raw.to_csv('raw_data/raw_tthc.csv', index=False)

In [13]:
link_raw.info()
link_raw.tail(1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9452 entries, 0 to 9451
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   link            9452 non-null   object
 1   phanLoai        9452 non-null   object
 2   boNganh         9452 non-null   object
 3   cauHoi          9452 non-null   object
 4   cauTraLoi       9449 non-null   object
 5   TTHCLienQuan    9452 non-null   object
 6   cauHoiLienQuan  9452 non-null   object
dtypes: object(7)
memory usage: 517.0+ KB


,link,phanLoai,boNganh,cauHoi,cauTraLoi,TTHCLienQuan,cauHoiLienQuan
9451,https://dichvucong.gov.vn/p/home/dvc-chi-tiet-...,[],[],"$questionInfo[""CONTENT""]","$i[""CONTENT""]",[],[]


In [14]:
tthc_raw.info()
tthc_raw.tail(1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2695 entries, 0 to 2694
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   link                    2695 non-null   object
 1   maThuTuc                2695 non-null   object
 2   soQuyetDinh             2615 non-null   object
 3   tenThuTuc               2615 non-null   object
 4   capThucHien             2615 non-null   object
 5   loaiThuTuc              2615 non-null   object
 6   linhVuc                 2615 non-null   object
 7   trinhTuThucHien         2695 non-null   object
 8   cachThucThucHien        2695 non-null   object
 9   thanhPhanHoSo           2695 non-null   object
 10  doiTuongThucHien        2615 non-null   object
 11  coQuanThucHien          2695 non-null   object
 12  coQuanCoThamQuyen       2615 non-null   object
 13  diaChiTiepNhanHoSo      2615 non-null   object
 14  coQuanDuocUyQuyen       2615 non-null   object
 15  coQu

,link,maThuTuc,soQuyetDinh,tenThuTuc,capThucHien,loaiThuTuc,linhVuc,trinhTuThucHien,cachThucThucHien,thanhPhanHoSo,...,coQuanThucHien,coQuanCoThamQuyen,diaChiTiepNhanHoSo,coQuanDuocUyQuyen,coQuanPhoiHop,ketQuaThucHien,canCuPhapLy,yeuCauDieuKienThucHien,tuKhoa,moTa
2694,https://dichvucong.gov.vn/p/home/dvc-chi-tiet-...,2.000895,1792 /QĐ-BKHCN,"Thủ tục đánh giá, nghiệm thu và công nhận kết ...",Cấp Tỉnh,TTHC không được luật giao cho địa phương quy đ...,Sở hữu trí tuệ,- Bước 1: Tổ chức chủ trì và chủ nhiệm dự án t...,"Hình thức nộp Thời hạn giải quyết Phí, lệ phí ...","Bao gồm\nTên giấy tờ Mẫu đơn, tờ khai Số lượng...",...,Sở Khoa học và Công nghệ,Không có thông tin,Sở Khoa học và Công nghệ,Không có thông tin,Không có thông tin,Quyết định công nhận kết quả thực hiện dự án c...,Số ký hiệu Trích yếu Ngày ban hành Cơ quan ban...,Không,Không có thông tin,Không có thông tin
